
# Förbehandling för maskinläsning
## Text
Vi börjar med någon form av definition av text: En text är en uppsättning *karaktärer*. Karaktärer utgörs av bokstäver, men också symboler som & eller ', och punktuering såsom ., :, -. Även mellanslag, " ", utgör en karaktär, men den brukar vanligen ha sin egen kategori.

En text är vanligtvis uppbyggd av *ord* åtskilda av mellanslag och punktuering. Varje ord i sin tur är en sammansättning *bokstäver*. Gemener och versaler utgör olika karaktärer, det vill säga "A" är inte samma karaktär som "a". Alfabetet existerar alltså i två uppsättningar.

Människan är en tolkningsmässig best som kan tolka text och koppla den till verkliga referenter. Detta görs både genom bokstavering, läsning av enskilda bokstäver för att forma ett ord (för nya begrepp), och symbolisk igenkänning av hela ord och fraser. Vi kan bortse från små och stora bokstäver och automatiskt åtgärda stavfel i text. Stora delar av den mänskliga läsförståelsen är därmed kontextuell och inferentiell. 


## När datorn skall läsa...
En dator har emellertid ingen uppfattning om sin omvärld och kan utan mänsklig hjälp endast memorisera. Den ser ingen betydelseskillnad på punkt och komma, på konsonanter och vokaler eller ens enskilda bokstäver. För datorn är allting bara karaktärer.

En betydande konsekvens av maskinens tillkortakommanden är att orden "Pelle", "Pwlle" och "pelle" är fundamentalt olika - de innehåller alla olika karaktärer. Det blir vidare ännu värre, då även "Pelle.", " Pelle ", "Pelle," och "Pelle:" är olika med den bokstavstroende maskinens ögon.

Detta gäller också språkligt naturliga variationer som böjningar och flektioner - människor identifierar enkelt att *lugn*, *lugna*, *lugnade*, *lugnar*, *lugnande* alla är ord med samma konnotation och nästan samma denotation. Alla dessa variationer är dock orelaterade för maskinen - de innehåller ju inte samma karaktärer. Det är dessutom mindre skillnad i karaktärer mellan *lugn* och *ugn* än *lugn* och *lugnar*, men endast det senare paret är relaterat. Obegripligt, tycker datorn.

## Informationsinnehåll
Människan har därför ett ansvar att underlätta för maskinen för maskinläsning. Det är därför vanligt att man försöker utvinna så mycket information som möjligt ur en text, och minska antalet variationer. I fallet *lugn-Lugn-lugn.-Lugn.* delar alla orden samma innehållsliga information, och i fallen *lugna-lugnar-lugnade* åtminstone samma grundbetydelse. För en dator utgör dessa varianter en form av brus, mindre avvikelser från den grundläggande betydelsen som har med *lugn* att göra. 

<div>
<img src="media/pipeline.png" width="450"/>
</div>


## En pipeline för förbehandling
Syftet med denna notebook är att ge datorn underlag för att känna igen betydelser i text, inte bara se denna som samlingar av karaktärer. Det är vanligt att text förbehandlas utifrån ett recept, vanligtvis kallat en *pipeline*. Processerna i pipeline kallas ibland *transformer* och utgörs vanligen av

- **Punktuering**: Ta bort all form av punktuering i datan (punkt, komma, semikolon, bindestreck, et c.)
- **Tokenisering**: Gör om texter till listor av ord, och ta bort alla mellanslag
- **Lemmatisering**: Transformera orden till sin oböjda grundform.
- **Filtrering**: Filtrering av oönskade ord, exempelvis sökord, lågfrekventa ord, eller väldigt så kallade stoppord.

Ordningen på dessa är essentiell, men inte satt i sten - man kan variera ordningen litet beroende på vilket resultat man är ute efter.

När en text blivit förbehandlad kan vi gå till nästa steg och representera texten numeriskt.

Vi börjar med att läsa in bibliotek för vissa funktioner vi behöver

In [2]:
import course.preprocess

[nltk_data] Downloading package punkt to /home/vws/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vws/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vws/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Datan
Exemplen i detta moment kommer att vara uteslutande på svenska, men den datan som används kommer dock att vara på engelska. Svenska är i språkteknologiska sammanhang ett lågresursspråk, där det till skillnad från storspråken engelska, spanska, kinesiska, tyska o.s.v. saknas många verktyg. 

In [4]:
data = course.preprocess.example()
data

"The definition of the digital humanities is being continually formulated by scholars/practitioners. Since the field is constantly growing and changing, specific definitions can \n        quickly become outdated or unnecessarily limit future potential.[4] The second volume of Debates \n        in the Digital Humanities (2016) acknowledges the difficulty in defining the field: 'Along with the \n        digital archives, quantitative analyses, and tool-building projects that once characterized the \n        field, DH now encompasses a wide range of methods and practices: visualizations of large image \n        sets, 3D modeling of historical artifacts, 'born digital' dissertations, hashtag activism and the \n        analysis thereof, alternate reality games, mobile makerspaces, and more. In what has been called \n        'big tent' DH, it can at times be difficult to determine with any specificity what, precisely, digital \n        humanities work entails.'[5] Historically, the digital h

## Punktuering

Punktuering introducerar brus i texten, som beskrivet tidigare: I meningsutbytet

- *Hej! Mitt namn är Leia, hur är det?*
- *Hej, det är bra, Leia.*

kommer alltså (*Hej!*, *Hej,*), (*Leia,*, *Leia.*) och (*det?*, *det*) alla att räknas som olika ord, vilket är oönskat. Punktuering har också relativt hög frekvens och därmed stor inverkan. 

Genom att ta bort punktuering så försvinner dock också information, syntax, meningsbyggnad, pauser, tilltal och liknande som används av mänskliga läsare. Det är dock värt att observera att punktuering inte är nödvändig för mänsklig läsning - många språk klarar sig med minimal punktuering i skrift.

Nedan skript tar bort punktuering, förutom snedstreck, som byts ut mot mellanslag, och bindestreck som byts ut mot understreck. På så vis bevarar vi självständiga ord som separeras av snedstreck ("smör/margarin") och namn med bindestreck ("Jean-Pierre").

In [3]:
data.remove_punctuation()

'The definition of the digital humanities is being continually formulated by scholars practitioners Since the field is constantly growing and changing specific definitions can \n        quickly become outdated or unnecessarily limit future potential 4  The second volume of Debates \n        in the Digital Humanities  2016  acknowledges the difficulty in defining the field Along with the \n        digital archives quantitative analyses and tool_building projects that once characterized the \n        field DH now encompasses a wide range of methods and practices visualizations of large image \n        sets 3D modeling of historical artifacts born digital dissertations hashtag activism and the \n        analysis thereof alternate reality games mobile makerspaces and more In what has been called \n        big tent DH it can at times be difficult to determine with any specificity what precisely digital \n        humanities work entails 5  Historically the digital humanities developed out of

## Tokenisering

Tokenisering avser att man delar upp strängar av text i betydelsebärande enheter. Det som vanligtvis avses är att texter delas upp i ord, *lexem*, men det behöver inte vara fallet. Det finns två huvudsakliga typer av tokenisering, som tillåter mer avancerade förfiningar.

Tokeniseringsmodeller har mycket gemensamt med vad som kallas *språkmodeller* (eng: *language models*) som försöker kodifiera hur språk genereras matematiskt eller kognitivt. Tänker vi på ordnivå eller på bokstavsnivå?


### Ordtokenisering
Texten delas upp vid mellanslag, så att varje *token* blir ett ord. En viss fras representeras då inte längre av en text, utan som en unik lista av ordtokens. Se exempelvis

```

"Längtan är en lek. När den växer till allvar, kallas den ångest." →

["Längtan", "är", "en", "lek.", "När", "den", "växer", "till", "allvar,", "kallas", "den", "ångest."]


```

där ``Längtan``, ``lek.`` och så vidare är tokens. Observera att då vi endast delat texten vid mellanslag, så räknas även punktuering in i föregående ord. Av de orsaker vi listade i föregående avsnitt brukar tokenisering alltså föregås av att man tar bort punktueringen.

Vad är fördelarna med ordtokenisering? Det finns lingvistiska argument för att stor del av vår förståelse sker på ordnivå, även om vad som faktiskt är ett "ord" är väldigt diskutabelt. Det är oavsett otvivelaktigt att ord bär *semantisk information*, de är alltså betydelsebärande. En applikation som är intresserad av att utvinna semantisk information bör alltså lämpligen ha tokenisering på ordnivå. Ett stort tillkortakommande är dock att en sådan tokenisering har mycket kort *räckvidd* - ordtokenisering kan inte representera uttryck som t. ex. partikelverb som *ge upp* och *ge med* eller fasta uttryck som *ge fyr*, där ordet *ge* inte har samma denotation. 

Man **tappar alltså också mycket information** vid tokenisering.


### Karaktärstokenisering
Vid karaktärstokenisering delas varje text upp mellan varje karaktär. På samma text som innan erhåller vi alltså

```

"Längtan är en lek. När den växer till allvar, kallas den ångest." →

["L", "ä", "n", "g", "t", "a", "n", " ", "e", "n", " ", "l", "e", "k", ".", " ", "N", "ä", "r", " ", "d", "e", "n", " ", "v", "ä", "x", "e", "r", " ", "t", "i", "l", "l", " ", "a", "l", "l", "v", "a", "r", ",", " ", "k", "a", "l", "l", "a", "s", " ", "d", "e", "n", " ", "å", "n", "g", "e", "s", "t", "."]


```

där det är viktigt att observera att både mellanslag `` `` och punktuering alltså räknas som karaktärer. I relation till ordtokeniseringen så kan karaktärstokenisering te sig ointuitiv och oanvändbar. Karaktärer bär alltså i allmänhet inte semantisk information (``L`` har ingen betydelse). Den har också mycket sämre räckvidd än ordnivån. Dock innehåller karaktärstokenisering information om frekvensen av vissa tecken, som kan vara väldigt användbart för att exempelvis identifiera en författares stil eller en viss texttyp.

In [4]:
data.tokenize()

['The', 'definition', 'of', 'the', 'digital', 'humanities', 'is', 'being', 'continually', 'formulated', 'by', 'scholars', 'practitioners', 'Since', 'the', 'field', 'is', 'constantly', 'growing', 'and', 'changing', 'specific', 'definitions', 'can', 'quickly', 'become', 'outdated', 'or', 'unnecessarily', 'limit', 'future', 'potential', '4', 'The', 'second', 'volume', 'of', 'Debates', 'in', 'the', 'Digital', 'Humanities', '2016', 'acknowledges', 'the', 'difficulty', 'in', 'defining', 'the', 'field', 'Along', 'with', 'the', 'digital', 'archives', 'quantitative', 'analyses', 'and', 'tool_building', 'projects', 'that', 'once', 'characterized', 'the', 'field', 'DH', 'now', 'encompasses', 'a', 'wide', 'range', 'of', 'methods', 'and', 'practices', 'visualizations', 'of', 'large', 'image', 'sets', '3D', 'modeling', 'of', 'historical', 'artifacts', 'born', 'digital', 'dissertations', 'hashtag', 'activism', 'and', 'the', 'analysis', 'thereof', 'alternate', 'reality', 'games', 'mobile', 'makerspace

## Normalisering
Normalisering kan också kallas *kanonisering* och har syftet att transformera texten till en konsekvent kanonform. Detta inbegriper egentligen nästan hela pipelinen för NLP, men vanligtvis åsyftas några specifika transformer. Vanliga normaliseringar är

- **Gemenisering**: Alla tokens skrivs med små bokstäver. På detta vis räknas tokens som ``Längta`` och ``längta`` som likadana. Finns det något problem med detta? 
- **Stavningsnormalisering**: Vanligt i engelska och språk med många sätt att skriva samma sak - tokens skrivs konsekvent enligt en stavning, exempelvis ``colour``/``color``, ``favourize``/``favourise``/``favorize``. Detta är också en nödvändighet i historiska svenska texter, där många ord hade flera giltiga stavningar (t. ex. ``dufva``/``duva``/``dwfa``). Här gäller även att konvertera numeriska uttryck till uttalsmotsvarigheter, som ``200 kr`` till ``tvåhundra kronor``. Det är ofta lättare att transformera de numeriska uttrycken än deras uttal, då de senare kan ha större variation.
- **Sammansättningar**: På engelska skrivs sammansatta ord vanligtvis inte med bindestreck eller ihop, utan med mellanslag. Ett exempel är ``data science`` eller ``peanut butter``. Dessa kan då skrivas ihop med ett understreck ``_`` eller liknande behandling för att se till att de behåller sin semantiska information.

Detta gäller nästan uteslutande ordtokenisering. I karaktärstokenisering är det oftast tillräckligt att gemenisera tecknen.

In [5]:
data.to_lower_case()

['the', 'definition', 'of', 'the', 'digital', 'humanities', 'is', 'being', 'continually', 'formulated', 'by', 'scholars', 'practitioners', 'since', 'the', 'field', 'is', 'constantly', 'growing', 'and', 'changing', 'specific', 'definitions', 'can', 'quickly', 'become', 'outdated', 'or', 'unnecessarily', 'limit', 'future', 'potential', '4', 'the', 'second', 'volume', 'of', 'debates', 'in', 'the', 'digital', 'humanities', '2016', 'acknowledges', 'the', 'difficulty', 'in', 'defining', 'the', 'field', 'along', 'with', 'the', 'digital', 'archives', 'quantitative', 'analyses', 'and', 'tool_building', 'projects', 'that', 'once', 'characterized', 'the', 'field', 'dh', 'now', 'encompasses', 'a', 'wide', 'range', 'of', 'methods', 'and', 'practices', 'visualizations', 'of', 'large', 'image', 'sets', '3d', 'modeling', 'of', 'historical', 'artifacts', 'born', 'digital', 'dissertations', 'hashtag', 'activism', 'and', 'the', 'analysis', 'thereof', 'alternate', 'reality', 'games', 'mobile', 'makerspace

In [6]:
data.add_compounds([('digital', 'humanities'), 
                    ('alternate', 'reality'), 
                    ('virtual', 'reality'), 
                    ('cultural', 'analytics'),
                    ('topic', 'modeling')])

['the', 'definition', 'of', 'the', 'digital_humanities', 'is', 'being', 'continually', 'formulated', 'by', 'scholars', 'practitioners', 'since', 'the', 'field', 'is', 'constantly', 'growing', 'and', 'changing', 'specific', 'definitions', 'can', 'quickly', 'become', 'outdated', 'or', 'unnecessarily', 'limit', 'future', 'potential', '4', 'the', 'second', 'volume', 'of', 'debates', 'in', 'the', 'digital_humanities', '2016', 'acknowledges', 'the', 'difficulty', 'in', 'defining', 'the', 'field', 'along', 'with', 'the', 'digital', 'archives', 'quantitative', 'analyses', 'and', 'tool_building', 'projects', 'that', 'once', 'characterized', 'the', 'field', 'dh', 'now', 'encompasses', 'a', 'wide', 'range', 'of', 'methods', 'and', 'practices', 'visualizations', 'of', 'large', 'image', 'sets', '3d', 'modeling', 'of', 'historical', 'artifacts', 'born', 'digital', 'dissertations', 'hashtag', 'activism', 'and', 'the', 'analysis', 'thereof', 'alternate_reality', 'games', 'mobile', 'makerspaces', 'and'

## Lemmatisering

Den kanske mest situationsberoende transformen är *lemmatisering*. I detta steg omvandlas specifikt ordtokens till sina *lemma* (pl. *lemmata*). Ett lemma är vad som brukar kallas ordboksformen av ett ord (ett lexem). Ordboksformen är den oböjda eller oflektionerade formen av ordet. Observera att vissa lexem är homonymer, det vill säga, låter och ser likadana ut, som *en val* och *ett val*. De har olika böjningslexem, och eftersom de har olika betydelse har de separata plats i ordboken. Deras lemma är dock detsamma, nämligen ``val``.

- Lexem: ``har``, ``hade``, ``haft``
    - Lemma: ``ha``
- Lexem: ``lekar``, ``lekarnas``, ``leks``
    - Lemma: ``lek``
- Lexem: ``leker``, ``lekte``, ``lekande``, ``leks``
    - Lemma: ``leka``
- Lexem: ``gå``, ``gick``, ``gått``
    - Lemma: ``gå``

Observera att lemmata är begränsade inom sin ordklass, och inte tar avledningar i beaktande. Trots att ``lek`` <=> ``leka`` har de olika lemma.

Lemmatisering är den absolut viktigaste transformen för en maskin. Denna process reducerar ner ``Han går till skolan`` och ``Han gick till skolan`` till ``Han gå till skolan``, och förenar därmed semantiken i båda fraserna. Medan (svensktalande) människor omedelbart känner igen att fraserna beskriver samma situation, så när som på tempus, kan maskiner endast dra den slutsatsen om fraserna har samma tokens.

Vad vinner vi? Genom lemmatisering förenar vi olika semantiker till en grundbetydelse som är tolkningsbar för datorn. Emellertid förlorar vi även här information - i ovan gick information om när händelsen skedde förlorad. Vidare kräver lemmatisering en väldigt omfattande referensordbok som kan identifiera lemma, vilket inte finns för alla språk, i synnerhet lågresursspråk.

### Alternativ: Stemming

Ett annat alternativ till lemmatisering är stemming, som till skillnad från lemmatisering inte kräver en referensordbok. Denna metod hittar istället stammen till ordtokens. Stammen är den (inledande) del av ordet som förblir oförändrad när detta böjs, exempelvis stammen ``jaga`` till ``jagade``, ``jagat``, ``jagas``, eller stammen ``gruv`` till ``gruva``, ``gruvor``, ``gruvornas``.   

I ovan fall hade vi istället fått stammarna

- Lexem: ``har``, ``hade``, ``haft``
    - Stam: ``ha``
- Lexem: ``lekar``, ``lekarnas``, ``leks``
    - Stam: ``lek``
- Lexem: ``leker``, ``lekte``, ``lekande``, ``leks``
    - Stam: ``lek``
    
Detta fungerar utmärkt i språk som använder sig av ändelser, exempelvis germanska språk som svenska och engelska. Men detta är en halvsanning - germanska språk har gott om starka böjningar, såsom i sista exemplet ger flera stammar:

- Lexem: ``gå``, ``gick``, ``gått``
    - Stam: ``gå``, ``gick``, ``gå``
    
Det finns många andra språk där detta blir ett fullständigt omöjligt företag, exempelvis arabiska.

In [7]:
data.lemmatize()

['the', 'definition', 'of', 'the', 'digital_humanities', 'is', 'being', 'continually', 'formulated', 'by', 'scholar', 'practitioner', 'since', 'the', 'field', 'is', 'constantly', 'growing', 'and', 'changing', 'specific', 'definition', 'can', 'quickly', 'become', 'outdated', 'or', 'unnecessarily', 'limit', 'future', 'potential', '4', 'the', 'second', 'volume', 'of', 'debate', 'in', 'the', 'digital_humanities', '2016', 'acknowledges', 'the', 'difficulty', 'in', 'defining', 'the', 'field', 'along', 'with', 'the', 'digital', 'archive', 'quantitative', 'analysis', 'and', 'tool_building', 'project', 'that', 'once', 'characterized', 'the', 'field', 'dh', 'now', 'encompasses', 'a', 'wide', 'range', 'of', 'method', 'and', 'practice', 'visualization', 'of', 'large', 'image', 'set', '3d', 'modeling', 'of', 'historical', 'artifact', 'born', 'digital', 'dissertation', 'hashtag', 'activism', 'and', 'the', 'analysis', 'thereof', 'alternate_reality', 'game', 'mobile', 'makerspaces', 'and', 'more', 'in

## Filtrering
### Stopp-ord

Det vanligtvis slutgiltiga steget utgör att manuellt ta bort ord man som forskare misstänker är högfrekventa men bär väldigt lite semantisk mening. Dessa kallas vanligen *stopp-ord*. På svenska är dessa vanligen så kallade funktionsord, såsom *men*, *att*, *ett*, *en*, *och*. 

Genom att filtrera bort dessa försöker man minimera hur mycket tid maskinen spenderar på att tolka närmast betydelselöst brus, för att istället koncentrera sig på tokens med större semantisk information. 


### Lågfrekventa tokens
Det är ofta vettigt att ta bort de minst förekommande tokens i texten. Dessa är ofta ord som endast förekommer någon enstaka gång, såsom namn ``Pelle``, ``Andersson``, ``Tapetföretaget AB`` eller facktermer. Motiveringen är att dessa oftast inte *bär* det semantiska innehållet i tillräckligt långa texter, utan istället utgör outliers maskinen kämpar med att förstå. Detta är högst situationsbetingat, och om syftet är att utreda relationer mellan individer i en text, kan sådan filtrering vara olämplig. 

Vidare är det olämpligt om texterna är särskilt korta (mindre än 1000 ord). Detta då frekvenser inte är stabila för särskilt små antal (jämför en opinionsundersökning på 15 person jämfört med en på 10000 personer).  

In [8]:
data.remove_stopwords(['ha'])

['definition', 'digital_humanities', 'continually', 'formulated', 'scholar', 'practitioner', 'since', 'field', 'constantly', 'growing', 'changing', 'specific', 'definition', 'quickly', 'become', 'outdated', 'unnecessarily', 'limit', 'future', 'potential', '4', 'second', 'volume', 'debate', 'digital_humanities', '2016', 'acknowledges', 'difficulty', 'defining', 'field', 'along', 'digital', 'archive', 'quantitative', 'analysis', 'tool_building', 'project', 'characterized', 'field', 'dh', 'encompasses', 'wide', 'range', 'method', 'practice', 'visualization', 'large', 'image', 'set', '3d', 'modeling', 'historical', 'artifact', 'born', 'digital', 'dissertation', 'hashtag', 'activism', 'analysis', 'thereof', 'alternate_reality', 'game', 'mobile', 'makerspaces', 'ha', 'called', 'big', 'tent', 'dh', 'time', 'difficult', 'determine', 'specificity', 'precisely', 'digital_humanities', 'work', 'entail', '5', 'historically', 'digital_humanities', 'developed', 'humanity', 'computing', 'ha', 'become'

In [9]:
data.remove_low_frequency_tokens(frequency=0.01)

['definition', 'digital_humanities', 'field', 'definition', 'become', 'digital_humanities', 'field', 'digital', 'analysis', 'project', 'field', 'dh', 'large', 'set', 'digital', 'analysis', 'game', 'ha', 'dh', 'digital_humanities', 'digital_humanities', 'humanity', 'computing', 'ha', 'become', 'field', 'computing', 'social', 'computing', 'medium', 'study', 'digital_humanities', 'data', 'mining', 'large', 'cultural', 'data', 'set', 'digital_humanities', 'humanity', 'cultural', 'study', 'social', 'science', 'computing', 'data', 'information', 'data', 'mining', 'mining', 'digital', 'digital', 'related', 'digital_humanities', 'study', 'study', 'study', 'field', 'digital_humanities', 'medium', 'study', 'information', 'science', 'medium', 'game', 'study', 'related', 'digital_humanities', 'project']

## Sammanfattningsvis

In [10]:
data = (course.example()
              .remove_punctuation()
              .tokenize()
              .to_lower_case()
              .add_compounds([('digital', 'humanities'), 
                    ('alternate', 'reality'), 
                    ('virtual', 'reality'), 
                    ('cultural', 'analytics'),
                    ('topic', 'modeling')])
              .lemmatize()
              .remove_stopwords(['ha'])
       )